# Build Action Corpus

We'll now build our corpus which stores actions that our bot can do. We
want to use Vectara as a recommendation engine for our Webhook when we
ask for a different task. We'll also use this to guide responses when the user
first starts chatting.

For this corpus we'll put in the following fields:
* **action-key:** This is our "key" which will trigger different actions within our Webhook
* **hidden:** Some actions should not be explicitly shown to a user, such as small talk
* **name:** This is the "human friendly" name for the action, which should be listed when the user first enters.

Each action will have a description of what it is to do - if the user enters a phrase which is similar, the system will ask the user whether they want to do "X" action.

## Initialize our Client
Once again, we initialize our client using implicit configuration.

We also initialize our logger and mute the OAuthUtil.

In [ ]:
from vectara_client.core import Factory
from vectara_client.admin import CorpusBuilder
import logging

logging.basicConfig(format='%(asctime)s:%(name)-35s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S %z')
logging.getLogger("OAuthUtil").setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

client = Factory().build()
manager = client.corpus_manager


## Create the Corpus
We'll again use our CorpusBuilder to create our corpus.

In [ ]:
corpus = (CorpusBuilder("meetup-actions")
         .description("Provides appropriate actions for user content using recommendations")
         .add_attribute("action-key", "This is our \"key\" which will trigger different actions within our Webhook")
         .add_attribute("hidden", "Whether the action is hidden from the end user")
         .add_attribute("name","This is the \"human friendly\" name for the action, which should be listed when the user first enters.")
         .build())
corpus_id = manager.create_corpus(corpus, delete_existing=True)

## Load Corpus with our Actions

In [ ]:
from pathlib import Path
import json

with open(Path("../resources/actions/bot-tasks.json"), "r") as f:
    for line in f.readlines():
        action = json.loads(line)
        logger.info(f"Found line:\n{json.dumps(action, indent=4)}")

        metadata = {
            "action-key": action["key"],
            "name": action["task"],
            "hidden": action["hidden"]
        }
        metadata_json = json.dumps(metadata)
        
        to_index = {
          "document_id": action["name"],
          "title": action["name"],
          "metadata_json": metadata_json,
          "section": [
            {
              "text": action["description"]
            }
          ]
        }

        client.indexer_service.index_doc(corpus_id, to_index)

## Test the Actions
Lets confirm the "top" actions for different user prompts and confirm our
recommendation engine is working as expected.

In [ ]:
def check_action(query, expected_action=None, min_score=0):
    
    responses = client.query_service.query(query, corpus_id, semantics="RESPONSE",summary=False)

    response = responses.response[0]
    
    document_index = response.documentIndex
    document = responses.document[document_index]
    
    action = ""
    for metadata in document.metadata:
        if metadata.name == "action-key":
            action = metadata.value
    
    logger.info(f"For user text, [{query}], Score [{response.score}], Action [{action}]")
    if expected_action and action != expected_action:
        raise Exception(f"We were expecting action [{expected_action}] but received [{action}]")
    if response.score < min_score:
        raise Exception(f"The action found was too weakly matched [{response.score}] and did not meet our minimun score of [{min_score}]")

# Tests with expected actions and minimum score
check_action("I would like to submit my talk")
check_action("I would like to submit my talk", "meetup-submission", .99)
check_action("Can we talk about DataEngBytes", "general", .99)
check_action("What else can you do?", "overview", .80)
